# Чат-бот "Кошка-из-трешки"

# Подключение API Yandex GPT

In [1]:
import os

import requests

api_key = "AQVN0-FV0kOBiKSo26Wi-jZCLPPoOdyBjGHs3pXV"
os.environ["api_key"] = api_key
api_key = os.environ["api_key"]
folder_id = "b1gst3c7cskk2big5fqn"

tg_notifications_api_token = "7007888308:AAHnHfCIlM3GyG0IftxXvIJvM2pdfzcRyRw"
tg_notifications_chat_id = "469119744"


def send_tg_message(text="Вычисление завершено"):
    requests.post(
        "https://api.telegram.org/"
        + "bot{}/sendMessage".format(tg_notifications_api_token),
        params=dict(chat_id=tg_notifications_chat_id, text=text),
    )

In [2]:
send_tg_message("Test message")

# 1. Обработка ссылок сайта priem.mai.ru

Изначально через сторонний сервис был получен список страниц сайта priem.mai.ru. Список записан в файл pages.txt.

Помещаем ссылки на все странички из текстового файла pages.txt в словарь `pages`:

In [ ]:
import logging

import requests
from bs4 import BeautifulSoup

# Настройка логирования
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger()

# Заголовки для HTTP-запроса
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
}


# Функция для получения заголовка страницы по URL
def get_page_title(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Проверка на успешный ответ
        soup = BeautifulSoup(response.text, "html.parser")
        title = soup.title.string if soup.title else "No title found"
        print(f"Успешно получен заголовок для {url}")
        return title
    except requests.RequestException as e:
        print(f"Ошибка при получении заголовка для {url}: {e}")
        return f"Error: {e}"


# Чтение ссылок из файла
with open("/content/drive/MyDrive/pages.txt", "r") as file:
    links = file.read().splitlines()
    print(f"Прочитано {len(links)} ссылок из файла")

# Создание словаря ссылок и заголовков страниц
pages = {}
for link in links:
    print(f"Обработка ссылки: {link}")
    title = get_page_title(link)
    pages[link] = title
    if "Error:" in title:
        print(f"Не удалось получить заголовок для {link}")
    else:
        print(f'Заголовок для {link}: "{title}"')

# Вывод результатов
# for link, title in pages.items():
#     print(f'{link}: {title}')

# Итоговый вывод
successful = sum(1 for title in pages.values() if not title.startswith("Error:"))
failed = len(pages) - successful
print(f"Успешно получено заголовков: {successful}")
print(f"Не удалось получить заголовков: {failed}")

Записываем список страниц и их заголовков в файл pages.json:

In [ ]:
import json

with open("/content/drive/MyDrive/pages.json", "w", encoding="utf-8") as f:
    json.dump(pages, f, ensure_ascii=False)

Для дальнейшей работы создадим необходимые директории:

In [ ]:
import os


def create_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
        print(f"Папка '{directory_name}' успешно создана.")
    else:
        print(f"Папка '{directory_name}' уже существует.")


create_directory("/content/drive/MyDrive/html")
create_directory("/content/drive/MyDrive/text")

Теперь организуем скачивание всех страничек на диск в формате HTML с помощью команды wget:

In [ ]:
import json
import os

from tqdm import tqdm

# Путь к файлу со списком ссылок в формате JSON
links_file = "/content/drive/MyDrive/pages.json"

# Директория для хранения загруженных HTML файлов
html_dir = "/content/drive/MyDrive/html"
os.makedirs(html_dir, exist_ok=True)

# Загрузка списка ссылок из файла JSON
with open(links_file, "r", encoding="utf-8") as f:
    urls = json.load(f)

# Загрузка страниц с использованием wget
for url in tqdm(urls, desc="Downloading pages"):
    # Создаем имя файла на основе URL
    filename = os.path.join(html_dir, url.split("/")[-1] + ".html")
    command = f'wget -O "{filename}" "{url}"'
    os.system(command)

print("Download complete.")

Download complete.


Далее переименуем названия страниц в соответствии с их заголовком

In [ ]:
import os
import string

from bs4 import BeautifulSoup


def rename_html_files(source_directory, target_directory):
    # Проверяем, существует ли целевая директория, если нет - создаем ее
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    # Проходим по всем файлам в исходной директории
    for filename in os.listdir(source_directory):
        if filename.endswith(".html") or filename.endswith(".htm"):
            source_filepath = os.path.join(source_directory, filename)

            # Открываем файл и читаем его содержимое
            with open(source_filepath, "r", encoding="utf-8") as file:
                content = file.read()

                # Используем BeautifulSoup для парсинга HTML
                soup = BeautifulSoup(content, "html.parser")

                # Извлекаем заголовок страницы
                title = soup.title.string if soup.title else "untitled"

                # Создаем новое имя файла из заголовка страницы
                # Удаляем недопустимые символы для имен файлов
                valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
                new_filename = (
                    "".join(
                        c
                        for c in title
                        if c in valid_chars or c.isalpha() or c.isspace()
                    )
                    .strip()
                    .replace(" ", "_")
                )

                # Добавляем расширение .html к новому имени файла
                new_filepath = os.path.join(target_directory, new_filename + ".html")

                # Переименовываем файл и копируем его в целевую директорию
                with open(new_filepath, "w", encoding="utf-8") as new_file:
                    new_file.write(content)
                print(f"Copied and renamed '{filename}' to '{new_filename}.html'")


# Пути к исходной и целевой папкам
source_directory_path = "/content/drive/MyDrive/html"
target_directory_path = "/content/drive/MyDrive/html_new"

rename_html_files(source_directory_path, target_directory_path)

### Извлекаем текст из страниц

Для извлечения текста из страничек, мы будем использовать библиотеку [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/https://beautiful-soup-4.readthedocs.io/en/latest/).

In [ ]:
import os
import re
import string

from bs4 import BeautifulSoup


def sanitize_filename(filename):
    # Удаляем недопустимые символы
    filename = re.sub(r'[\\/*?:"<>|]', "", filename)
    # Ограничиваем длину имени файла до 100 символов
    return filename[:100]


def rename_html_files(source_directory, target_directory):
    # Проверяем, существует ли целевая директория, если нет - создаем ее
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    # Проходим по всем файлам в исходной директории
    for filename in os.listdir(source_directory):
        if filename.endswith(".html") or filename.endswith(".htm"):
            source_filepath = os.path.join(source_directory, filename)

            # Пытаемся открыть файл с кодировкой utf-8
            try:
                with open(source_filepath, "r", encoding="utf-8") as file:
                    content = file.read()
            except UnicodeDecodeError:
                print(f"Skipping file due to encoding issues: {filename}")
                continue

            # Используем BeautifulSoup для парсинга HTML
            soup = BeautifulSoup(content, "html.parser")

            # Извлекаем заголовок страницы
            title = soup.title.string if soup.title else "untitled"

            # Создаем новое имя файла из заголовка страницы
            # Удаляем недопустимые символы для имен файлов
            valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
            new_filename = (
                "".join(
                    c for c in title if c in valid_chars or c.isalpha() or c.isspace()
                )
                .strip()
                .replace(" ", "_")
            )

            # Санируем новое имя файла
            new_filename = sanitize_filename(new_filename)

            # Добавляем расширение .html к новому имени файла
            new_filepath = os.path.join(target_directory, new_filename + ".html")

            # Проверяем, существует ли уже файл с таким именем, если да, добавляем суффикс
            counter = 1
            original_new_filepath = new_filepath
            while os.path.exists(new_filepath):
                new_filepath = f"{original_new_filepath[:-5]}_{counter}.html"
                counter += 1

            # Переименовываем файл и копируем его в целевую директорию
            with open(new_filepath, "w", encoding="utf-8") as new_file:
                new_file.write(content)
            print(f"Copied and renamed '{filename}' to '{new_filename}.html'")


# Укажите пути к исходной и целевой папкам
source_directory_path = "/content/drive/MyDrive/html"
target_directory_path = "/content/drive/MyDrive/html_new"

rename_html_files(source_directory_path, target_directory_path)

Теперь пройдёмся по всем страничкам в директории `content/html`, и запишем в директорию `content/text` текстовые эквиваленты.

In [ ]:
import os
import re

from bs4 import BeautifulSoup
from tqdm import tqdm


# Функция для извлечения текста и заголовка из HTML с отладочной информацией
def get_text(html):
    soup = BeautifulSoup(html, "html.parser")

    # Ищем заголовок страницы
    title_element = soup.find("title")
    if not title_element:
        print("Title element not found")
        print(
            soup.prettify()[:1000]
        )  # Печатаем первые 1000 символов HTML для диагностики
    title = title_element.text if title_element else "No title found"

    # Ищем основной контент страницы
    body_element = soup.find("div", {"class": "page-wrapper"})
    if not body_element:
        print("Body element not found")
        print(
            soup.prettify()[:1000]
        )  # Печатаем первые 1000 символов HTML для диагностики
    body = body_element.text if body_element else "No content found"

    # Собираем полный текст страницы
    text = f"{title}\n{body}"
    return title, text


# Полный путь к директории с HTML файлами
html_dir = "/content/drive/MyDrive/html_new"
# Создание директории для текстовых файлов
text_dir = "/content/drive/MyDrive/text"
os.makedirs(text_dir, exist_ok=True)

# Процесс извлечения текста и записи его в файлы
for fn in tqdm(os.listdir(html_dir)):
    if fn.endswith(".html"):  # Проверка расширения файла
        filepath = os.path.join(html_dir, fn)
        with open(filepath, "r", encoding="utf-8") as file:
            html = file.read()

        try:
            title, all_text = get_text(html)
        except Exception as e:
            print(f"Error processing {fn}: {e}")
            continue

        # Замена нескольких подряд идущих пустых строк на одну
        all_text = re.sub(r"[\r\n]{4,}", r"\r\n", all_text)
        text_filename = os.path.join(text_dir, f"{fn[:-5]}.txt")

        with open(text_filename, "w", encoding="utf-8") as f:
            f.write(f"## {title}\n")
            f.write(all_text)

print("Processing complete.")

  1%|          | 6/629 [00:01<02:15,  4.60it/s]

Body element not found
<!DOCTYPE html>
<html lang="ru">
 <head>
  <title>
   IT-магистратура в Москве
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="#f5f5f5" name="theme-color"/>
  <meta content="website" property="og:type"/>
  <meta content="IT-магистратура в Москве" property="og:title"/>
  <meta content="https://priem.mai.ru//it/" property="og:url"/>
  <meta content="1323748498089307" property="fb:app_id"/>
  <meta content="IT-магистратура в Москве" property="twitter:title"/>
  <meta content="@MAIuniversity" property="twitter:site"/>
  <meta content="summary_large_image" property="twitter:card"/>
  <meta content="Все образовательные программы IT-магистратуры МАИ. Московский авиационный институт входит в тройку лидеров ВУЗов, выпускающих на рынок труда самых востребованных специалистов в сфере информационных технологий согласно рейтингу HeadHunter." property="og:description"/>
  <meta content="Все образоват

  7%|▋         | 42/629 [00:08<00:54, 10.76it/s]

Body element not found
<!DOCTYPE html>
<html lang="ru">
 <head>
  <title>
   Календарь абитуриента
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="#f5f5f5" name="theme-color"/>
  <meta content="website" property="og:type"/>
  <meta content="Календарь абитуриента" property="og:title"/>
  <meta content="https://priem.mai.ru//calendar/" property="og:url"/>
  <meta content="1323748498089307" property="fb:app_id"/>
  <meta content="Календарь абитуриента" property="twitter:title"/>
  <meta content="@MAIuniversity" property="twitter:site"/>
  <meta content="summary_large_image" property="twitter:card"/>
  <meta content="День открытых дверей, сроки приёма, мастер-классы, университетские субботы" property="og:description"/>
  <meta content="День открытых дверей, сроки приёма, мастер-классы, университетские субботы" property="twitter:description"/>
  <meta content="Календарь абитуриента" property="og:image:alt"/>
  <me

100%|██████████| 629/629 [01:25<00:00,  7.32it/s]

Processing complete.


Теперь запишем содержимое `class_info` в отдельные файлы, начинающиеся с `class_`. Эти тексты тоже могут нам пригодиться для формирования ответов.

In [ ]:
class_info = set()

d = {}
for x in class_info:
    name = x.split("\r")[0].split("\n")[0]
    if name not in d:
        d[name] = x
    else:
        if len(x) < len(d[name]):
            d[name] = x
for k, v in d.items():
    with open(f"/content/drive/MyDrive/text/class_{k}.txt", "w", encoding="utf-8") as f:
        f.write(v)

# 2. Строим вопрос-ответного бота по технологии Retrieval-Augmented Generation на LangChain

## Разбиваем документ на части

Для работы retrival augmented generation нам необходимо по запросу найти наиболее релевантные фрагменты исходного текста, на основе которых будет формироваться ответ. Для этого нам надо разбить текст на такие фрагменты, по которым мы сможем вычислять эмбеддинг, и которые будут с запасом помещаться во входное окно используемой большой языковой модели.

Для этого можно использовать механизмы фреймворка LangChain - например, `RecursiveCharacterTextSplitter`. Он разбивает текст на перекрывающиеся фрагменты по набору типовых разделителей - абзацы, переводы строк, разделители слов.

#### Тестовая версия (обрабатываются первые 5 файлов)

In [2]:
import langchain
import langchain.document_loaders
from langchain.text_splitter import RecursiveCharacterTextSplitter

user = "rep-res"
chunk_size = 1024
chunk_overlap = 50
source_dir = "text"

# Загрузчик документов из директории
loader = langchain.document_loaders.DirectoryLoader(
    source_dir, glob="*.txt", show_progress=True, recursive=True
)

# Загрузка всех документов
all_documents = loader.load()

# Ограничение на первые 20 документов
documents = all_documents[:5]

# Разделитель текста на фрагменты
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Разделение отфильтрованных документов на фрагменты
# fragments = splitter.create_documents([x.page_content for x in documents])
fragments = splitter.split_documents(documents)

print(f"Общее количество фрагментов: {len(fragments)}")

100%|█████████▉| 629/630 [02:00<00:00,  5.21it/s]

Общее количество фрагментов: 32


#### Полная версия (обрабатываются все файлы)

In [12]:
import langchain
import langchain.document_loaders as loaders
from langchain.text_splitter import RecursiveCharacterTextSplitter

user = "rep-res"
chunk_size = 1024
chunk_overlap = 50
source_dir = "text"

# Загрузчик документов из директории
loader = loaders.DirectoryLoader(
    source_dir, glob="*.txt", show_progress=True, recursive=True
)

# Разделитель текста на фрагменты
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Загрузка и разделение текстов на фрагменты
documents = loader.load()
# fragments = splitter.create_documents([doc.page_content for doc in documents])
fragments = splitter.split_documents(documents)

print(f"Общее количество фрагментов: {len(fragments)}")

100%|██████████| 629/629 [02:05<00:00,  5.03it/s]


Общее количество фрагментов: 9683


## Вычисляем эмбеддинги для всех фрагментов

Для вычисления эмбеддингов берем модель из репозитория HuggingFace под названием distiluse-base-multilingual-cased-v1:

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

# Инициализация модели эмбеддингов
embeddings = HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v1")

# Вычисление эмбеддингов для всех фрагментов с прогресс-баром
sample_vec = embeddings.embed_query("Hello, world!")
len(sample_vec)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


512

Также пробуем использовать более продвинутую модель [эмбеддингов от Yandex GPT](https://cloud.yandex.ru/docs/yandexgpt/api-ref/Embeddings/). Для работы с ней используем библиотеку [yandex_chain](http://githib.com/yandex-datasphere/yandex-chain), поскольку соответствующая поддержка пока не появилась в LangChain.

In [4]:
from yandex_chain import YandexEmbeddings
from chromadb import Documents, EmbeddingFunction, Embeddings

# Инициализация модели эмбеддингов Yandex GPT
embeddings = YandexEmbeddings(folder_id=folder_id, api_key=api_key)

# Вычисление эмбеддингов
vec = embeddings.embed_query("Hello, world!")
sample_vector_length = len(embeddings.embed_query("Hello World"))
print(f"Sample vector length for Yandex: {sample_vector_length}")

Sample vector length for Yandex: 256


In [16]:
import os
from yandex_chain import YandexEmbeddings
from chromadb import Documents, EmbeddingFunction, Embeddings
from langchain.embeddings import OpenAIEmbeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, folder_id, api_key):
        self.embeddings = YandexEmbeddings(folder_id=folder_id, api_key=api_key)

    def __call__(self, input: Documents) -> Embeddings:
        return [self.embeddings.embed_query(doc) for doc in input]
    
    def embed_documents(self, documents: Documents) -> Embeddings:
        return self.__call__(documents)
    
    def embed_query(self, query: str) -> Embeddings:
        return self.embeddings.embed_query(query)

embedding_function = MyEmbeddingFunction(folder_id, api_key)

### Cохраняем эмбеддинги  в векторную БД

Для поиска нам нужно уметь быстро сопоставлять эмбеддинг запроса, и эмбеддинги всех фрагементов наших исходных материалов. Для этого используются векторные базы данных. Для нашего примера мы используем небольшую векторную БД [LanceDB](https://lancedb.com/), хранящую индекс в директории на диске.

Первоначально создадим базу данных, добавив туда одну строчку:

In [6]:
%pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.3 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 49.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00

In [18]:
import os

import lancedb
from langchain.vectorstores import LanceDB

db_dir = "store"

os.makedirs(db_dir, exist_ok=True)

db = lancedb.connect(db_dir)
table = db.create_table(
    "vector_index",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

In [17]:
import os

from langchain.vectorstores import Chroma

# Инициализация директории для базы данных
db_dir = "store"

os.makedirs(db_dir, exist_ok=True)

# Создание базы данных и добавление начальных данных
db = Chroma(
    persist_directory=db_dir,
    embedding_function=embedding_function  # Передаем функцию для вычисления эмбеддингов
)

db.add_texts(
    texts=["Hello World"],
    metadatas=[{"id": "1"}],
    ids=["1"],
    embeddings=[embeddings.embed_query("Hello World")],
)

# Сохранение изменений
db.persist()

In [19]:
# Проверка корректного создания базы данных и добавленных данных
def check_database(db):
    # Попробуем извлечь данные по тексту "Hello World"
    results = db.similarity_search("Hello World", k=1)
    if results and results[0].page_content == "Hello World":
        print("Initial data check passed.")
    else:
        print("Initial data check failed.")

    # Проверим количество записей в базе данных
    total_records = db._collection.count()
    print(f"Total records in the database: {total_records}")

    # Проверка на присутствие всех фрагментов
    for fragment in fragments:
        results = db.similarity_search(fragment, k=1)
        if results and results[0].page_content == fragment:
            print(f"Fragment '{fragment}' check passed.")
        else:
            print(f"Fragment '{fragment}' check failed.")

check_database(db)

Initial data check passed.
Total records in the database: 1


UnboundLocalError: local variable 'js' referenced before assignment

In [12]:
embeddings_list = [
    embeddings.embed_query(text) for text in fragments
]  # Получение эмбеддингов для каждого фрагмента

# Подключение к существующей базе данных
db = Chroma(persist_directory=db_dir)

# Добавление новых данных
db.add_texts(texts=fragments, embeddings=embeddings_list)

# Сохранение изменений
db.persist()

UnboundLocalError: local variable 'js' referenced before assignment

Далее проиндексируем все выделенные ранее фрагменты нашей документации, используя реализованный нами выше адаптер для YandexGPT-эмбеддингов.

In [19]:
# db = LanceDB.from_documents(fragments, embeddings, connection=db)
db = LanceDB.from_documents(fragments, embeddings)

[2024-05-25T08:30:44Z WARN  lance::dataset] No existing dataset at /tmp/lancedb/vectorstore.lance, it will be created


In [111]:
print(lancedb.__version__)

0.6.8


In [20]:
q = "Какие направления обучения есть в МАИ?"
res = db.similarity_search(q)
print(res[0].page_content)

## Выбираем направления обучения в МАИ | новости МАИ Выбираем направления обучения в МАИ | новости МАИ Московский авиационный институт Приёмная комиссия Предыдущий Предприём заявлений

Как поступить?

Подготовительные курсы

Приказы о зачислении Cледующий Контакты Меню для мобильных телефонов Меню ГлавнаяСтатьиВыбираем направления обучения в МАИ

Выбираем направления обучения в МАИ | новости МАИ

Выбираем направления обучения в МАИ | новости МАИ 19 мая 2022

В МАИ состоялся цикл мероприятий, направленных на знакомство абитуриентов с конкурсными группами и программами обучения.

Руководители институтов и ответственные секретари рассказали, чему учат на каждом из меганаправлений, об образовательных и карьерных траекториях, целевой подготовке, практике и стажировках и внеучебной жизни.


Также в качестве пример попробуем использовать еще один полезный интерфейс для поиска текстов - `Retriever`, убедимся, что он тоже работает:

In [21]:
retriever = db.as_retriever(search_kwargs={"k": 5})
res = retriever.get_relevant_documents(q)
for x in res:
    print(x.page_content)

## Выбираем направления обучения в МАИ | новости МАИ Выбираем направления обучения в МАИ | новости МАИ Московский авиационный институт Приёмная комиссия Предыдущий Предприём заявлений

Как поступить?

Подготовительные курсы

Приказы о зачислении Cледующий Контакты Меню для мобильных телефонов Меню ГлавнаяСтатьиВыбираем направления обучения в МАИ

Выбираем направления обучения в МАИ | новости МАИ

Выбираем направления обучения в МАИ | новости МАИ 19 мая 2022

В МАИ состоялся цикл мероприятий, направленных на знакомство абитуриентов с конкурсными группами и программами обучения.

Руководители институтов и ответственные секретари рассказали, чему учат на каждом из меганаправлений, об образовательных и карьерных траекториях, целевой подготовке, практике и стажировках и внеучебной жизни.
4. Подготовительные курсы МАИ

Центр «Подготовительные курсы МАИ» осуществляет подготовку учащихся 8-11 классов к поступлению в университет и их дальнейшему обучению в МАИ и других вузах. Занятия по подгото

## Подключаемся к Yandex GPT

Для формирования целостного ответа на вопрос пользователей нам потребуется обработать найденные фрагменты текста с помощью большой языковой модели. Фреймворк LangChain поддерживает интеграцию с различными большими языковыми моделями, включая Yandex GPT.

In [122]:
from yandex_chain import YandexLLM

instructions = """
Ты - дружелюбный чат-бот, отвечающий на вопросы абитуриентов по поступлению в МАИ
"""

llm = YandexLLM(api_key=api_key, folder_id=folder_id, instruction_text=instructions)
llm(q)

send_tg_message()

В данном примере мы пока что никак не использовали наши текстовые документы.

## Собираем Retrieval-Augmented Generation

Пришла пора соединить всё вместе и научить бота отвечать на вопросы, подглядывая в наш текстовый корпус. Для этого используем механизм цепочек (*chains*). Основную функциональность реализует `StuffDocumentsChain`, которая делает следующее:

1. Берёт коллекцию документов `input_documents`
1. Каждый из них пропускает через `document_prompt`, и затем объединяет вместе.
1. Данный текст помещается в переменную `document_variable_name`, и передаётся большой языковой модели `llm`

В нашем случае `document_prompt` не будет модицицировать документ, а основной запрос к LLM будет содержать в себе инструкции для Yandex GPT.

In [23]:
import langchain.chains
import langchain.prompts

# Промпт для обработки документов
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
Текст:
-----
{context}
-----
Вопрос:
{query}"""
prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Создаём цепочку
llm_chain = langchain.chains.LLMChain(llm=llm, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
chain.run(input_documents=res, query=q)

/home/jupyter/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'В тексте представлена следующая информация о направлениях обучения в **Московском авиационном институте (национальном исследовательском университете):**\n1. Базовое высшее.  \n2. Специализируемое высшее.  \n3. Среднее профессиональное.\n\n**Также в тексте упоминается о меганаправлениях:**\n1. *«Технопредпринимательство*», предполагающее подготовку предпринимателей, создающих инновационные стартапы в различных отраслях технологий и нацеленных на коммерциализацию своих разработок.\n2. *«Инженер будущего*», обеспечивающее глубокое погружение в инженерную деятельность посредством включения в образовательный процесс элементов научно-исследовательской работы, проектной деятельности, а также прохождения практик и стажировок.\n3. *«Интеллектуальные авиационные системы*». \n\nБолее точная информация представлена на официальном сайте МАИ.'

Чтобы ещё более улучшить результат, мы можем использовать хитрый механизм перемешивания документов, таким образом, чтобы наиболее значимые документы были ближе к началу запроса. Также мы оформим все операции в одну функцию `answer`:

In [24]:
from langchain.document_transformers import LongContextReorder

reorderer = LongContextReorder()


def answer(query, reorder=True, print_results=False):
    results = retriever.get_relevant_documents(query)
    if print_results:
        for x in results:
            print(f"{x.page_content}\n--------")
    if reorder:
        results = reorderer.transform_documents(results)
    return chain.run(input_documents=results, query=query)

In [1]:
answer("Перечисли все меганаправления в МАИ", print_results=False)

NameError: name 'answer' is not defined

Можно сравнить результаты, выдаваемые Yandex GPT напрямую, с результатами нашей генерации на основе документов:

In [26]:
def compare(q):
    print(f"Ответ YGPT: {llm(q)}")
    print(f"Ответ бота: {answer(q)}")


compare("Какие направления есть в МАИ?")

Ответ YGPT: В Московском авиационном институте (национальном исследовательском университете) на 2023 год доступно большое число различных направлений подготовки высшего образования. Вот некоторые из них:
* «Авионика и применяемая авионика»;
 * «Аэронавигация и эксплуатация авиационной техники»;
* «Двигатели летательных аппаратов»; 
 * «Инновационные технологии проектирования металлических, композиционных и неметаллических материалов для изделий авиационно-космической техники»; 
*  «Информационно-аналитические системы и технологии»; 

* «Интеллектуальные системы компьютерной навигации и управления летательными аппаратами»;
*  «Инновационные методы конструирования и технологии в машиностроении»;
*«Инновационный менеджмент в авиастроении»; 
*«Конструктивно-технологическое проектирование ракет-носителей и космических аппаратов»;*
*  и другие.
По каждому направлению магистратуры доступны различные профили, интересующие абитуриента, можно изучить на странице направления в официальном сайте М

# 3. QA

In [ ]:
import json
import os
import re
import time
from glob import glob

from grpc._channel import _MultiThreadedRendezvous
from langchain.llms import YandexGPT
from langchain.output_parsers.json import SimpleJsonOutputParser
from tqdm import tqdm

gpt = YandexGPT(api_key=os.environ["api_key"], folder_id=folder_id)

prompt = """
Тебе нужно прочитать текстовый файл с информацией с сайта приемной комиссии Московского авиационного института (МАИ). На основе этой информации тебе необходимо составить 5 вопрос-ответных пар в формате JSON. Вопросы должны быть подробными и конкретными, и охватывать все ключевые аспекты информации, представленной в файле, а ответы должны быть точными и информативными. Формат вопрос-ответных пар должен быть следующим:
[
    {"request":"вопрос 1","response":"ответ 1"},
    {"request":"вопрос 2","response":"ответ 2"},
    {"request":"вопрос 3","response":"ответ 3"},
    ...
]

Убедись, что каждый вопрос точно соответствует информации, представленной в тексте, и что ответы содержат всю необходимую информацию для корректного ответа на вопрос.

Ниже приведена доступная информация:
```
{PAGE}
```
"""

jp = SimpleJsonOutputParser()
gpt.temperature = 0.1

# Словарь для хранения вопрос-ответных пар
pairs = {}


def process_files(fns):
    for fn in tqdm(fns, desc="Processing files"):
        if fn in pairs.keys():
            print(f"Already processed: {fn}")
            continue
        print(f"Processing {fn}")
        text = open(fn, "r", encoding="utf-8").read()
        p = prompt.replace("{PAGE}", text)
        try:
            res = gpt(p)
            # print("RES:\n", res)
        except _MultiThreadedRendezvous as e:
            print(f"Error processing {fn}")
            continue

        # Предварительная обработка для удаления символов перевода строки и экранирования кавычек
        res = re.sub(r"\\n", " ", res)
        res = re.sub(r"(\r\n|\n|\r)", " ", res)
        res = re.sub(r'"', r'\\"', res)
        res = res.replace('\\"', '"')
        res = res.replace("*", "")

        # print("RES v2:\n", res)

        # Используем регулярное выражение для поиска JSON объектов
        json_matches = re.findall(r"\{.*?\}", res, re.DOTALL)
        data = []
        for match in json_matches:
            try:
                obj = json.loads(match)
                data.append(obj)
            except json.JSONDecodeError as e:
                print(f"Ошибка декодирования JSON в строке: {match}\n{e}")

        pairs[fn] = data
        save_to_json(pairs, output_file)
        print(f"Pairs {fn}\n: {pairs[fn]}")
        time.sleep(1)


# Функция для сохранения результатов в JSON файл
def save_to_json(pairs, output_file):
    all_qa_pairs = []
    for fn, qa_list in pairs.items():
        all_qa_pairs.extend(qa_list)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(all_qa_pairs, f, ensure_ascii=False, indent=4)


# Запуск обработки файлов
file_list = glob("/home/jupyter/datasphere/project/text/*")
process_files(file_list)

Processing files:   0%|          | 0/629 [00:00<?, ?it/s]

Processing /home/jupyter/datasphere/project/text/23 абитуриента_сетевых_программ_МАИ_и СмолГУ_подписали_договоры_с авиационным_заводом__новости_МАИ.txt
Pairs /home/jupyter/datasphere/project/text/23 абитуриента_сетевых_программ_МАИ_и СмолГУ_подписали_договоры_с авиационным_заводом__новости_МАИ.txt
: [{'request': 'Какие специальности будут доступны для бюджетного обучения в сентябре 2023 года?', 'response': 'По специальностям «Проектирование, производство и эксплуатация ракет и ракетно-космических комплексов» и «Самолёто- и вертолётостроение»'}, {'request': 'Сколько человек заключили целевые договоры с авиационным заводом?', 'response': '23 абитуриента'}, {'request': 'Сколько человек поступят на бюджетное обучение в сентябре 2023 года?', 'response': '50 человек'}, {'request': 'Какие специальности доступны для платного обучения?', 'response': 'Все специальности, кроме «Проектирование, производство и эксплуатация ракет и ракетно-космических комплексов» и «Самолёто- и вертолётостроение»'},

Processing files:   0%|          | 1/629 [00:25<4:23:27, 25.17s/it]

Processing /home/jupyter/datasphere/project/text/58 маёвцев_прошли_практику_на Комсомольском-на-Амуре_авиационном_заводе__новости_МАИ.txt
Pairs /home/jupyter/datasphere/project/text/58 маёвцев_прошли_практику_на Комсомольском-на-Амуре_авиационном_заводе__новости_МАИ.txt
: []


Processing files:   0%|          | 2/629 [00:50<4:23:47, 25.24s/it]

...

Processing /home/jupyter/datasphere/project/text/Иностранным_гражданам_1.txt
Pairs /home/jupyter/datasphere/project/text/Иностранным_гражданам_1.txt
: [{'request': 'Какие документы необходимы для поступления в МАИ?', 'response': 'Для поступления в МАИ необходимо предоставить следующие документы: - заявление о приёме; - документ, удостоверяющий личность, гражданство и возраст (или их копии); - документ об образовании и (или) о квалификации; - фотографии; - документы, подтверждающие особые права и преимущества при поступлении; - иные документы, предусмотренные законодательством Российской Федерации и локальными нормативными актами МАИ. '}, {'request': 'Какие вступительные испытания проводятся в МАИ самостоятельно?', 'response': 'В МАИ самостоятельно проводятся следующие вступительные испытания: - вступительные испытания по общеобразовательным предметам; - вступительные испытания по дополнительным вступительным испытаниям творческой и (или) профессиональной направленности; - вступительные

Processing files:  23%|██▎       | 143/629 [1:01:01<3:29:54, 25.91s/it]

Processing /home/jupyter/datasphere/project/text/Иностранным_гражданам_2.txt
Pairs /home/jupyter/datasphere/project/text/Иностранным_гражданам_2.txt
: []


Processing files:  23%|██▎       | 144/629 [1:01:28<3:32:19, 26.27s/it]

Processing /home/jupyter/datasphere/project/text/Интегрированные_интеллектуальные_робототехнические_комплексы.txt


Теперь преобразуем всё к формату, в котором Yandex GPT принимает фразы для до-обучения:

In [102]:
res = []
stop = []
for _, p in pairs.items():
    for d in p:
        if "request" in d and "response" in d and not (d["request"].lower() in stop):
            res.append(d)

print(f"Всего {len(res)} вопрос-ответных пар")

Всего 1179 вопрос-ответных пар


Запишем результат на диск:

In [103]:
with open("pairs_for_train_ygpt.json", "w", encoding="utf-8") as f:
    json.dump(res, f, ensure_ascii=False)

# 4. Запуск бота в Telegram

In [129]:
%pip install nest_asyncio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


E0526 12:04:50.638531588  307331 backup_poller.cc:127]                 Run client channel backup poller: UNKNOWN:pollset_work {created_time:"2024-05-26T12:04:50.638402754+00:00", children:[UNKNOWN:Bad file descriptor {created_time:"2024-05-26T12:04:50.638305375+00:00", errno:9, os_error:"Bad file descriptor", syscall:"epoll_wait"}]}
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import asyncio

import nest_asyncio
from telegram import Update
from telegram.ext import (
    ApplicationBuilder,
    CallbackContext,
    CommandHandler,
    MessageHandler,
    filters,
)

# Вставьте здесь ваш токен API
TOKEN = "6560264857:AAGfINoUv8vwqEHQvjP3kRUT3SOAYzPcRiE"


async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text("Привет! Задай мне вопрос.")


async def help_command(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text("Напиши мне любой вопрос, и я постараюсь ответить.")


async def handle_message(update: Update, context: CallbackContext) -> None:
    user_question = update.message.text
    # Вызов функции для ответа на вопрос
    bot_response = answer(user_question, print_results=False)
    await update.message.reply_text(bot_response)


async def stop(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text("Бот остановлен.")
    await context.application.stop()
    await context.application.shutdown()


async def main() -> None:
    # Создайте приложение и передайте ему токен вашего бота.
    application = ApplicationBuilder().token(TOKEN).build()

    # Инициализация приложения
    await application.initialize()

    # На разных командах - ответите в Telegram
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(CommandHandler("stop", stop))  # Команда для остановки

    # На текстовые сообщения - ответите в Telegram
    application.add_handler(
        MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message)
    )

    # Запуск бота
    await application.start()
    await application.updater.start_polling()

    # Ожидание завершения работы
    await application.updater.idle()


# Активируем nest_asyncio
nest_asyncio.apply()

# Запуск бота в текущем цикле событий
if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())

In [ ]:
send_tg_message()